In [ ]:
!pip install transformers

In [ ]:
from transformers import PreTrainedTokenizerFast
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '</s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
# PreTrainedTokenizerFast 클래스의 from_pretrained 메소드를 사용하여 사전 훈련된 토크나이저를 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


['▁안녕',
 '하',
 '세',
 '요.',
 '▁한국어',
 '▁G',
 'P',
 'T',
 '-2',
 '▁입',
 '니다.',
 '😤',
 ':)',
 'l^o']

In [ ]:
import torch
from transformers import GPT2LMHeadModel # GPT2 LM Head Model : LM Head가 추가된 GPT-2 모델, 주로 자연어 생성 작업에 사용

# GPT2LMHeadModel 클래스의 from_pretrained 메소드를 사용하여 사전 훈련된 GPT-2 모델을 로드
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
# encode() : token string을 token id 의 리스트로 변환
# return_tensors : 토큰화된 결과를 파이썬 정수 목록 대신 텐서로 반환
# 'tf': TensorFlow tf.constant 객체
# 'pt': PyTorch torch.Tensor 객체
# 'np': Numpy np.ndarray 객체

gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)

# decode() : tokenizer 와 vocabulary를 이용해서 token id를 string으로 변환
generated = tokenizer.decode(gen_ids[0])
print(generated)

근육이 커지기 위해서는 무엇보다 규칙적인 생활습관이 중요하다.
특히, 아침식사는 단백질과 비타민이 풍부한 과일과 채소를 많이 섭취하는 것이 좋다.
또한 하루 30분 이상 충분한 수면을 취하는 것도 도움이 된다.
아침 식사를 거르지 않고 규칙적으로 운동을 하면 혈액순환에 도움을 줄 뿐만 아니라 신진대사를 촉진해 체내 노폐물을 배출하고 혈압을 낮춰준다.
운동은 하루에 10분 정도만 하는 게 좋으며 운동 후에는 반드시 스트레칭을 통해 근육량을 늘리고 유연성을 높여야 한다.
운동 후 바로 잠자리에 드는 것은 피해야 하며 특히 아침에 일어나면 몸이 피곤해지기 때문에 무리하게 움직이면 오히려 역효과가 날 수도 있다.
운동을


In [ ]:
import pandas as pd
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv",
    filename="ChatBotData.csv",
)
Chatbot_Data = pd.read_csv("ChatBotData.csv")
# print(Chatbot_Data.shape)

Chatbot_Data = Chatbot_Data[:1000]
Chatbot_Data.head()

# Q : 발화
# A : 발화
# label : 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [ ]:
from torch.utils.data import DataLoader, Dataset
import numpy as np
import re
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([?.!,])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([?.!,])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        # #질문의 길이가 최대길이보다 크면
        # if q_len > self.max_len:
        #     a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
        #     if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
        #         q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
        #         q_len = len(q_toked)
        #         a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
        #     a_toked = a_toked[:a_len]
        #     a_len = len(a_toked)

        # #질문의 길이 + 답변의 길이가 최대길이보다 크면
        # if q_len + a_len > self.max_len:
        #     a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
        #     if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
        #         q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
        #         q_len = len(q_toked)
        #         a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
        #     a_toked = a_toked[:a_len]
        #     a_len = len(a_toked)

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
model.to(device)
model.train()
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18
losses  = 0

train_set = ChatbotDataset(Chatbot_Data, max_len=40)

#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

print ("start")
for epoch in range(epoch):
    losses  = 0
    # train_dataloader에서 배치 단위로 샘플을 가져와 학습
    for batch_idx, samples in enumerate(train_dataloader):
        # Gradient 0으로 초기화
        optimizer.zero_grad()

        # 샘플에서 token_ids, mask, label을 가져옴
        token_ids, mask, label = samples
        token_ids = token_ids.to(device)
        mask = mask.to(device)
        label = label.to(device)

        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        # print(out.shape)
        # print(mask.shape)
        # print(mask.unsqueeze(dim=2).shape)

        # repeat_interleave : out의 2번째 차원의 크기만큼 반복(repeats=out.shape[2]), 2번째(dim=2) 차원을 따라 반복이 수행
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        # print(mask_3d.shape)

        # mask_3d가 1인 위치에는 출력 값을 그대로 사용,
        # mask_3d가 0인 위치에는 out 배열과 동일한 크기의 배열을 생성하고, 모든 원소의 값을 Sneg 값으로 설정
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))

        # mask_out 배열과 label 사용하여 손실 함수를 계산
        loss = criterion(mask_out.transpose(2, 1), label)

        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()

        # Backward pass (gradient 계산)
        avg_loss.backward()

        # Parameter update
        optimizer.step()

         # 손실 누적
        losses += avg_loss.item()
    print(f'epoch : %5d | loss : %.5f ' %(epoch+1, losses / len(list(train_dataloader))))
print ("end")

<ipython-input-8-af146c2e1177>:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


start
epoch :     1 | loss : 35.94357 
epoch :     2 | loss : 34.32792 
epoch :     3 | loss : 33.79182 
epoch :     4 | loss : 33.50476 
epoch :     5 | loss : 33.38018 
epoch :     6 | loss : 33.20537 
epoch :     7 | loss : 33.25392 
epoch :     8 | loss : 33.08635 
epoch :     9 | loss : 32.88683 
epoch :    10 | loss : 32.99221 
end


In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0).to(device)
            pred = model(input_ids)
            pred = pred.logits.to(device)
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().cpu().numpy().tolist())[-1]
            # PyTorch 텐서는 GPU 메모리에 저장될 수 있지만, NumPy 배열은 항상 호스트(CPU) 메모리에 저장된다.
            # 따라서, GPU 메모리에 저장된 PyTorch 텐서를 직접 NumPy 배열로 변환할 수 없다
            # numpy 메소드를 사용하려면 다음과 같이 처리해줘야한다
            # tensor = tensor.cpu().numpy()
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

Chatbot > 따뜻해졌죠
Chatbot > 연정이가요
Chatbot > 괜찮은 선택이길 바라요
Chatbot > 그 말을 한 사람이 가장 슬퍼요
Chatbot > 그렇게 될 수 있을 거예요
